In [37]:
import numpy as np
import pandas as pd
import re
import random
import time
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer

training_data = pd.read_excel("NPD_Data.xlsx", header=0).values
amazon = pd.read_excel("Amazon_Unmapped4.xlsx", header=0).values

In [38]:
# Divides mapped from unmapped
testing_data = amazon[amazon[:,3] != "Unmapped"]
amazon_unmapped = amazon[amazon[:,3] == "Unmapped"]

In [40]:
# Divides description of each product into individual words
def splitter(df):
    split = [0]*df.shape[0]
    # Useless strings to be removed
    purge_list = ["","a","and","with","for","in","the","of","to","x","disney","-","&"]
    
    # Divides description into individual strings
    for i in range(0,df.shape[0]):
        split[i] = [x.lower() for x in (re.split('[ _,+\'/#?:;()|]',str(df[i][0])))]
        
        for j in purge_list:
            while(j in split[i]): 
                split[i].remove(j)
    return split

In [41]:
training_corpus = splitter(training_data)
testing_corpus = splitter(testing_data)

In [42]:
# Not super useful, just counts occurences of each term in the entire sheet
def count_occurrences(df):
    dict = {}
    for i in range(0,len(df)):
        for j in range(0,len(df[i])):
            if df[i][j] not in dict:
                dict[df[i][j]] = {}
            if i not in dict[df[i][j]]:
                dict[df[i][j]][i] = 0
            dict[df[i][j]][i] += 1
    return dict

In [43]:
training_count = count_occurrences(training_corpus)

In [44]:
# Finds occurences of each property, e.g. "Frozen": 2000
def count_properties(dic):
    count = {}
    for i in dic:
        count[i] = 0
        for j in dic[i]:
            count[i] += dic[i][j]
    return count

In [45]:
CP = count_properties(training_count)
# Sorts dictionary of properties into order of occurrence
CP = {k: v for k, v in sorted(CP.items(), key=lambda item: item[1], reverse=True)}

In [46]:
# Lists to be used for training
X = training_data[:,1]
y = training_corpus

In [10]:
# This will take FOREVER, so just let it sit and run for a good 20mins(?)
count = 0
trainingDict = {}
# Go through all the properties and the list of products and add each indivdual word of item description to the property it associates with and tally the occurences of this word
# Word will then build an "identity of words that are commonly used with it.
load = len(CP)
prev = 0
start_time = time.time()
for listKeyword in CP:
    if round((count/load)*10) != prev:
            print(round((count/load)*10)*10, "% complete...")
            t = time.time() - start_time
            print("Elapsed time:", round((t)-(t%1))/60, "minutes\n")
            prev = round((count/load)*10)
    for j in range(len(y)):
        for keyword in y[j]:
            if listKeyword == keyword:
                if X[j] not in trainingDict:
                    trainingDict[X[j]] = {}
                if listKeyword not in trainingDict[X[j]]:
                    trainingDict[X[j]][listKeyword] = 1
                else:
                    trainingDict[X[j]][listKeyword] += 1
    count += 1

10 % complete...
Elapsed time: 3.6166666666666667 minutes

20 % complete...
Elapsed time: 10.8 minutes

30 % complete...
Elapsed time: 18.183333333333334 minutes

40 % complete...
Elapsed time: 25.816666666666666 minutes

50 % complete...
Elapsed time: 35.28333333333333 minutes

60 % complete...
Elapsed time: 47.0 minutes

70 % complete...
Elapsed time: 56.36666666666667 minutes

80 % complete...
Elapsed time: 66.11666666666666 minutes

90 % complete...
Elapsed time: 74.48333333333333 minutes

100 % complete...
Elapsed time: 82.81666666666666 minutes



In [48]:
for i in list(trainingDict):
    for j in list(trainingDict[i]):
        if trainingDict[i][j] == 1:
            del trainingDict[i][j]

del trainingDict["All Other"]
del trainingDict["Other License"]
# Number of unique properties used in training set
print(len(trainingDict))
pickle.dump(trainingDict, open("TrainingData.p","wb"))

# Will print FAT list of properties and list of words used with it. Just scroll past it.
# for i in trainingDict:
#     print(i)
#     print(trainingDict[i])
#     print("\n")

2704


In [49]:
trainingDict = pickle.load(open( "TrainingData.p","rb" ))

In [50]:
property_tree = [
    ["avengers","marvel","marvel universe","marvel other","spider-man","iron man","guardian of the galaxy","guardians of the galaxy","captain america movie 3","thor movie 3",
     "avengers movie 3 part 1","captain america movie - do not use","Spider-Man Movie (2017)","Ultimate Spider-Man","Thor - Do Not Use"],
    ["disney princess","frozen","disney frozen","frozen 2","frozen ii","beauty and the beast","little mermaid","cinderella","tangled","sleeping beauty","snow white & the 7 dwarfs",
     "Princess and the Frog","aladdin","Pocahontas","Mulan"],
    ["cars the movie","cars","cars 2","cars 3"],
    ["mickey mouse","mickey & friends","minnie","mickey mouse club"],
    ["star wars","star wars: rogue one","star wars - other","the mandalorian","star wars: the force awakens (episode 7)","The Phantom Menace (Episode 1)","The Empire Strikes Back (Episode 5)",
    "Star Wars Sequels - Other","Clone Wars"],
    ["frozen","disney frozen","frozen 2","frozen ii"],
    ["winnie the pooh","winnie the pooh & friends"],
    ["disney fairies","fairies"],
    ["finding nemo/finding dory","finding dory","finding nemo"],
    ["lilo & stitch","lilo and stitch"],
    ["nightmare before christmas","nightmare before xmas"],
    ["big hero 6","big hero six"],
    ["Lion Guard","The Lion Guard"],
    ["Monsters,  Inc.","Monsters Inc"],
    ["Moana/Vaiana","Moana"]
]
property_tree = [[x.lower() for x in sublist] for sublist in property_tree]

In [ ]:
def testRandom(prodSplit, propList, thres, shouldMult):
    correct = 0
    total = 0
#     print("Running test...")
    ignored = 0
    for i in range(2000):
        isCorrect = False
        index = random.randrange(len(propList))
        product = prodSplit[index]
#         print(product)
        top10 = {}
        top102 = {}
        followCheck = ""
        previousWord = ""
        for word in product:
#             print("Word: ", word)
            for property in trainingDict:
                for keyword in trainingDict[property]:
                    if word == keyword:
                        if property in top10:
                            if previousWord in trainingDict[property]:
                                if shouldMult: top10[property] += (trainingDict[property][keyword]*trainingDict[property][previousWord])-trainingDict[property][previousWord]
                                else: top10[property] *= trainingDict[property][keyword]
                            else:
                                top10[property] += trainingDict[property][keyword]
                        else:
                            top10[property] = trainingDict[property][keyword]

                        if property in top102:
                            top102[property] += 1
                        else:
                            top102[property] = 1
#                         print("Property: ", property, trainingDict[property][keyword])
            previousWord = word


        top10 = {k: v for k, v in sorted(top10.items(), key=lambda item: item[1], reverse=True)}
        top102 = {k: v for k, v in sorted(top102.items(), key=lambda item: item[1], reverse=True)}
        # ~40% of products are ignored
        if len(top10) < 2 or top10[next(iter(top10))]/top10[list(top10.keys())[1]] <= thres:
            ignored += 1
            continue
#         print(top10)
        total += 1
        if top10 != {}:
            if next(iter(top10)).lower() == propList[index].lower():
                correct += 1
                isCorrect = True
#                 print("Correct:", propList[index])
            else:
                for branch in property_tree:
                    if next(iter(top10)).lower() in branch and propList[index].lower() in branch:
                        correct += 1
                        isCorrect = True
#                         print("Correct:", next(iter(top10)), "&", propList[index])
                        break
                if not isCorrect:
#                     print(product)
#                     print("Incorrect:",next(iter(top10)),"\nAnswer:", propList[index], "\n")
#                     print("Correct? (t/f/na):")
#                     x = input()
#                     x = "f"
#                     if x == "t":
#                         correct += 1
#                         isCorrect = True
#                     elif x == "na":
#                         total -= 1
#                     else:
#                         print(top10, "\n")
                
    return ignored, correct, total

In [69]:
def predict(prodSplit):
    done = 0
    count = 0
    load = 500
    prev = 0
    start_time = time.time()
    
    propList = [0]*500
    print("Running prediction...")
    for i in range(500):
        
        if round((count/load)*10) != prev:
            print(round((count/load)*10)*10, "% complete...")
            t = time.time() - start_time
            print("Elapsed time:", round((t)-(t%1))/60, "minutes\n")
            prev = round((count/load)*10)
        index = random.randrange(len(prodSplit))
        product = prodSplit[index]
        top10 = {}
        top102 = {}
        followCheck = ""
        previousWord = ""
        for word in product:
            for property in trainingDict:
                for keyword in trainingDict[property]:
                    if word == keyword:
                        if property in top10:
                            if previousWord in trainingDict[property]:
                                # TODO: Must change so doesnt multiply into total property score but into previous word's score
                                top10[property] *= trainingDict[property][keyword]
                            else:
                                top10[property] += trainingDict[property][keyword]
                        else:
                            top10[property] = trainingDict[property][keyword]

                        if property in top102:
                            top102[property] += 1
                        else:
                            top102[property] = 1
            previousWord = word


        top10 = {k: v for k, v in sorted(top10.items(), key=lambda item: item[1], reverse=True)}
        top102 = {k: v for k, v in sorted(top102.items(), key=lambda item: item[1], reverse=True)}
        if len(top10) < 2 or top10[next(iter(top10))]/top10[list(top10.keys())[1]] <= 20: # This number will have to vary depending on size of data set
            propList[i] = [prodSplit[index],"Unmapped"]
            done += 1
            continue
        if top10 != {}:
            propList[i] = [prodSplit[index], next(iter(top10))]
        else:
            propList[i] = [prodSplit[index],"Unmapped"]
            done += 1
    print("Mapping complete.")
    print((load-done)*100/load, "% mapped")            
    return propList

In [62]:
# for i in [True, False]:
#     for j in range(5,105,5):
#         print(i,",", j,",", testRandom(testing_corpus,testing_data[:,3], j/10, i))
print(testRandom(testing_corpus,testing_data[:,3], 4, True))

KeyboardInterrupt: 

In [70]:
x = predict(testing_corpus)

Running prediction...
Mapping complete.
64.8 % mapped


In [72]:
for i in range(20):
    j = random.randrange(len(x))
#     if x[j][1] == "Unmapped": continue
    print(x[j],"\n")

[['mickey', 'mouse', 'pluto', 'goofy', 'donald', 'duck', 'american', 'flag', 'tee', 'shirt', 'small'], 'Mickey & Friends'] 

[['official', 'licensed', 'merchandise', 'lilo', 'stitch', 'vintage', 'stitch', 'unisex', 't-'], 'Lilo And Stitch'] 

[['ahgahg', '50pcs', 'kids', 'gift', 'party', 'boxes', 'little', 'mermaid', 'gift', 'bags', 'party', 'decora', 'an'], 'Unmapped'] 

[['baby', 'baby', 'minnie', 'projector'], 'Mickey & Friends'] 

[['mickey', 'mouse', 'pack', '10', 'bags', 'rectangular', 'maxi', 'clubhouse', '30', '40', 'cm', 'verb'], 'Mickey & Friends'] 

[['cars', 'maxi-colo', 'pixar'], 'Cars The Movie'] 

[['6.5cm', 'cinderella', 'snow', 'globe', 'beautiful', 'decoration', 'mi21'], 'Disney Princess'] 

[['muppets', 'it', 's', 'your', 'birthday', 'so', 'forget', 'about', 'past', 'forget', 'about', 'your', 'p'], 'Unmapped'] 

[['bulb', 'botz', 'bulbbotz', 'marvel', 'captain', 'america', 'alarm', 'clock'], 'Avengers'] 

[['marvel', 'avengers', 'super', 'heros', 'hulk', 'iron', 'man